### 结构化数据一般使用pandas中的DataFrame进行处理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow.keras import layers,models

In [ ]:
dftrain_raw=pd.read_csv('./data/train_csv')
dftest_raw=pd.read_csv('./data/test_csv')

In [ ]:
def preprocessing(data):
    df_result=pd.DataFrame()
    #类别性one-hot编码
    df_class=pd.get_dummies(data['Pclass'])   #编码之后为多列，因此，对两个dataframe 进行concat
    df_calss.columns=['Pclass_'+str(x) for x in df_class.columns]
    df_result=pd.concat([df_result,df_class],axis=1)
    #数值型特征，并对缺失值处理
    df_result['age']=data['age'].fillna(0)
    df-result['age_null']=pd.isna(data['age']).astype('int32')   #返回0，1
    

In [ ]:
x_train=preprocessing(dftrain_raw)
y_train=dftrain_raw['label'].values

x_test=preprocessing(dftest_raw)
y_test=dftest_raw['label'].values

print("x_train.shape = ",x_train.shape)
print("x_test.shape = ",x_test.shape)

## 定义模型

1. Sequential 按层顺序构建模型
2. 使用函数API构建任意结构模型
3. 继承Model类构建自定义模型

In [10]:
model=models.Sequential()
model.add(layers.Dense(20,activation='relu',input_shape(15,)))
model.add(layers.Dense(10,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()

## 训练模型
模型训练有三种方法，内置fit方法，内置train_on_batch方法，以及自定义训练循环

In [ ]:
#而分类问题选择二元交叉熵损失函数
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics='['AUC']'
             )
history=model.fit(x_train,y_train,
                 batch_size=64,
                 epochs=30,
                 validation_split=0.2) #划分训练集和测试机

## 评估模型

In [ ]:
import matplotlib.pyplot as plt
def plot_metric(history,metric):
    train_metrics=history.history[metric]
    val_metrics=history.history["val"+metric]
    epochs=ranges(1,len(trian_metrics)+1)
    plt.plot(epochs,train_metrics,'bo--')
    plt.plot(epochs,val_metrics,'ro--')
    plt.title('Training and validation'+metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legemd("train_"+metric,"val_"+metric)
    plt.show

In [ ]:
plot_metric(history,'loss')

In [ ]:
plot_metric(history,"AUC")

In [ ]:
# 查看模型在测试集上的效果
model.evaluate(x=x_test,y=y_test)

## 使用模型

In [ ]:
#预测概率
model.predict(x_test[0:10])
#预测类别
y_pred=model.predict_classes(x_test)

## 保存模型

可以使用Keras方式保存模型，也可以使用TensorFlow原生方式保存。前者仅仅适合使用Python环境恢复模型，后者则可以跨平台进行模型部署。

推荐使用后一种方式进行保存。

### 1.keras 方式保存模型

In [ ]:
#保存模型的结构及权重
model.save(./data/keras_model.h5)
del model #删除现有模型
#下载模型
model=models.load_model(./data/keras_model.h5)
model.evaluate(x_test,y_test)

In [ ]:
#保存模型结构
json_str=model.to_json()
#恢复模型结构
model_json=model.model_from_json(json_str)

In [ ]:
#保存模型权重
model.save_weights('./data/keras_model_weight.h5')
#恢复模型结构
model_json=models.model_from_json(json_str)
model_json.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['AUC'])
#加载权重
model_json.load_weights('./data/keras_model_weight.h5')
model_json.evaluate(x_test,y_test)

### TensorFlow原生方式保存

In [ ]:
# 保存模型结构与模型参数到文件,该方式保存的模型具有跨平台性便于部署

model.save('./data/tf_model_savedmodel', save_format="tf")
print('export saved model.')

model_loaded = tf.keras.models.load_model('./data/tf_model_savedmodel')
model_loaded.evaluate(x_test,y_test)